<a href="https://colab.research.google.com/github/keylacampusano/Python-Practice/blob/main/Embeddings_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb
!pip install sentence-transformers
!pip install faiss-cpu
!pip install datasets
!pip install pandas
!pip install spacy
!python -m spacy download en_core_web_sm


In [ ]:
!pip install nltk
!pip isntall gensim

In [3]:
#Hello, en esta parte vamos a importar Chroma DB, FAISS, y todas las librerias que utilizaremos

import chromadb
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
import spacy
from collections import defaultdict
from time import time
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from chromadb.config import Settings
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os


In [271]:
#Import Dataset from HF
dataset = load_dataset("embedding-data/sentence-compression")

In [272]:
df_train = pd.DataFrame(dataset["train"])


In [273]:
texts = df_train["set"]

In [274]:
print (texts)

0         [The USHL completed an expansion draft on Mond...
1         [Major League Baseball Commissioner Bud Selig ...
2         [It's fresh cherry time in Michigan and the be...
3         [An Evesham man is facing charges in Pennsylva...
4         [NRT LLC, one of the nation's largest resident...
                                ...                        
179995    [Before turning to questions of the soul, Mr P...
179996    [NBA Player Javale McGee dunks, blocks and the...
179997    [Saudi Arabia urged its citizens to avoid trav...
179998    [Auburn police are investigating the death of ...
179999    [Two of the most annoying forms of musical exp...
Name: set, Length: 180000, dtype: object


In [8]:
df_train.shape

(10, 1)

In [ ]:
#Now we import models from HF
from transformers import AutoTokenizer, AutoModel

model_st = SentenceTransformer('paraphrase-distilroberta-base-v1')



In [12]:
import concurrent.futures

In [193]:
embeddings = model_st.encode(texts)

In [196]:
def encode_texts(model):
    return model_st.encode(texts)

texts_embeddings = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Launch text encoding for each model in parallel to obtain a fastest result
    futures = [executor.submit(encode_texts, model) for model in model_st]

    # Gather results as they complete with the embed
    for future, model in zip(concurrent.futures.as_completed(futures), model_st):
        texts_embeddings.append(future.result())

In [195]:
print(embeddings)

[[-0.01476244 -0.11661881 -0.0585124  ... -0.07039721 -0.00150099
   0.3003174 ]
 [ 0.01026955  0.24575628 -0.01019975 ...  0.08323041  0.2253961
   0.2159326 ]
 [-0.17097703  0.24742264  0.3673481  ...  0.76167136  0.04267222
   0.10653625]
 ...
 [-0.2833038  -0.23699163 -0.0029812  ...  0.42410824 -0.07033559
  -0.10299055]
 [-0.2131704  -0.02782431 -0.09215224 ...  0.24908543  0.41691625
   0.24698234]
 [ 0.11760301  0.36261845 -0.04503371 ... -0.02317489  0.2086976
   0.10080846]]


In [197]:
embeddings.shape

(10, 768)

In [198]:
#def a function to search by similarity, (idea is to have an input and the sentence to compare, and then to obtain a similarity with the euclidan vector)
def euclidean_distances(X, Y=None, *, Y_norm_squared=None, squared=False, X_norm_squared=None):

  def search_similarity(query_vector, embeddings, distance_metric="euclidean", k=5):
    if distance_metric == "euclidean":
        distances = euclidean_distances(query_vector.reshape(1, -1), embeddings)
    else:
        raise ValueError("Unsupported distance metric")
    sorted_indices = np.argsort(distances)
    return sorted_indices[:k], distances[sorted_indices][:k]



In [201]:
#Now we need to create the indexes for FAISS & CHROMA DB, sirven para mejorar el rendimiento de las consultas de búsqueda por similitud.

vector_dimensions = embeddings.shape[1]

index= faiss.IndexFlatL2(vector_dimensions)
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [217]:
import numpy as np

search_text = 'its fresh'
search_vector = model_st.encode(search_text)
new_vector = np.array(search_vector.reshape(1, -1))
faiss.normalize_L2(new_vector)

In [218]:
distances,ann = index.search(new_vector, k=4)
results= pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
df_merged = pd.merge(results,df_train,left_on='ann', right_index=True)

In [245]:
df_merged.head()

,distances,ann,set
0,1.467611,2,[It's fresh cherry time in Michigan and the be...
1,1.807476,4,"[NRT LLC, one of the nation's largest resident..."
2,1.835946,7,"[The renovated Marappalam bridge, which had be..."
3,1.839991,0,[The USHL completed an expansion draft on Mond...


In [ ]:
#from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
!pip install scikit-learn


In [253]:
print(query_vector.shape)


(768,)


In [254]:
from sklearn.metrics.pairwise import cosine_similarity
def cosine_similarity(query_vector, embeddings, k=5):
  similarities = cosine_similarity(query_vector.reshape(embeddings.shape[0], -1), embeddings)
  sorted_indices = similarities.argsort()[0][::-1][:k]
  sorted_distances = similarities[0][sorted_indices]
  query_vector = query_vector.reshape(embeddings.shape[0], -1)
  return sorted_indices, sorted_distances

# Ejemplo de uso
search_text = 'it is fresh'
search_vector = model_st.encode(search_text)

# Realizar búsqueda utilizando la distancia coseno
similar_indices, similar_distances = cosine_similarity(search_vector, embeddings, k=5)

# Imprimir resultados
print("Índices de textos similares:", similar_indices)
print("Similitudes:", similar_distances)

ValueError: cannot reshape array of size 768 into shape (10,newaxis)

In [ ]:
!pip install chromadb
import chromadb

In [187]:
!apt-get install chromadb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package chromadb
